# Machine Learning CO<sub>2</sub> Working Capacity of MOFs

โดย รังสิมันต์ เกษแก้ว <br>
มหาวิทยาลัยแห่งซูริค <br>
E-mail: rangsiman1993@gmail.com

## 0. Import packages

In [21]:
# import standard scientific libraries
import os
import math
import numpy as np
import pandas as pd

# import ML models from scikit-learn
# from sklearn.linear_model import LinearRegression
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn import svm
# from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error

## 1. Import the data

In [22]:
pd.set_option('max_columns', None)
pd.set_option("display.precision", 8)

dataset = "./"

เนื่องจากในโปรเจ็คนี้เป็นการสาธิตการรัน Machine Learning ดังนั้นจึงจะขอดึงข้อมูล MOFs แค่ 20,000 โครงสร้างแรกเท่านั้น !!!

In [23]:
train = pd.read_csv(dataset + "new_train.csv")[:20000]
train.shape

(20000, 15)

In [24]:
train[train["surface_area"] < 0]

,MOFname,volume,weight,density,surface_area,void_fraction,void_volume,functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2,CO2_working_capacity
883,mof_unit_884,3084.500504,2201.75644,1.18531498,-1.0,-1.0,0.0,NO2-OH,4,13,21,acs,283.07423140,7.592979,315.21581447
1630,mof_unit_1631,2065.502764,1946.13554,1.56457593,-1.0,-1.0,0.0,OMe-NO2,4,1,9,acs,159.79438560,10.334164,154.62621208
2463,mof_unit_2464,4332.123412,2391.00639,0.91649322,-1.0,-1.0,0.0,I-NO2,4,17,25,acs,133.24169650,5.433424,185.84119686
2750,mof_unit_2751,6242.980835,3120.90776,0.83011502,-1.0,-1.0,0.0,NHMe-NO2,4,7,27,acs,154.55797600,5.612676,180.78457204
3376,mof_unit_3377,6266.570349,3616.97176,0.95843914,-1.0,-1.0,0.0,OEt-NO2,4,7,27,acs,304.26153490,6.251191,377.69844343
3845,mof_unit_3846,4114.140596,2880.33916,1.16255589,-1.0,-1.0,0.0,NH2-NO2,4,6,17,acs,232.39354680,6.559723,216.10121380
3930,mof_unit_3931,4014.788213,4205.48624,1.73941382,-1.0,-1.0,0.0,COOH-NO2,4,1,11,acs,180.99417660,11.017116,144.54729837
7385,mof_unit_7386,5149.814191,2703.22500,0.87164567,-1.0,-1.0,0.0,CN-NO2,4,6,7,acs,228.79968810,5.905715,222.57320494
8564,mof_unit_8565,2007.475254,1975.88672,1.63441065,-1.0,-1.0,0.0,NO2-NH2,4,1,22,acs,210.66731170,12.653423,185.47769830
9495,mof_unit_9496,1688.338733,1308.42068,1.28687735,-1.0,-1.0,0.0,NO2-Et,4,5,11,acs,147.89772630,7.059560,186.04638701


In [25]:
train.head()

,MOFname,volume,weight,density,surface_area,void_fraction,void_volume,functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2,CO2_working_capacity
0,mof_unit_1,1116.667429,875.240600,1.30152630,0.00,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.86416611,6.786041,105.28450172
1,mof_unit_2,2769.503842,2211.697211,1.32609012,603.61,0.13794,0.1040,F-OMe,10,44,57,etb,33.61678033,7.147286,101.22477418
2,mof_unit_3,1089.818728,773.687960,1.17885642,788.50,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.26372569,6.347967,118.98701075
3,mof_unit_4,2205.198301,1304.638720,0.98240815,1441.53,0.21814,0.2220,H-SO3H,9,17,24,sra,25.70137692,6.190085,187.62600450
4,mof_unit_5,1137.800963,901.736120,1.31602006,0.00,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.00183795,6.478063,79.21000066


In [27]:
feat = list(train.columns.values)
feat = {k: v for k, v in enumerate(feat)}
feat

{0: 'MOFname',
 1: 'volume',
 2: 'weight',
 3: 'density',
 4: 'surface_area',
 5: 'void_fraction',
 6: 'void_volume',
 7: 'functional_groups',
 8: 'metal_linker',
 9: 'organic_linker1',
 10: 'organic_linker2',
 11: 'topology',
 12: 'CO2/N2_selectivity',
 13: 'heat_adsorption_CO2',
 14: 'CO2_working_capacity'}

## 2. Clean data

In [28]:
train = train.iloc[:, [1,2,3,4,5,6,8,9,12,13,14]]
train

,volume,weight,density,surface_area,void_fraction,void_volume,metal_linker,organic_linker1,CO2/N2_selectivity,heat_adsorption_CO2,CO2_working_capacity
0,1116.667429,875.240600,1.30152630,0.00,0.07899,0.0607,3,4,22.86416611,6.786041,105.28450172
1,2769.503842,2211.697211,1.32609012,603.61,0.13794,0.1040,10,44,33.61678033,7.147286,101.22477418
2,1089.818728,773.687960,1.17885642,788.50,0.14874,0.1262,2,22,19.26372569,6.347967,118.98701075
3,2205.198301,1304.638720,0.98240815,1441.53,0.21814,0.2220,9,17,25.70137692,6.190085,187.62600450
4,1137.800963,901.736120,1.31602006,0.00,0.07778,0.0591,2,1,30.00183795,6.478063,79.21000066
...,...,...,...,...,...,...,...,...,...,...,...
19995,5189.923599,2423.541440,0.77542327,2944.18,0.33672,0.4342,12,11,8.85242576,4.787996,63.35786482
19996,13710.436745,4426.378000,0.53610092,3208.43,0.44798,0.8356,3,3,14.80568930,4.909318,90.66697340
19997,1215.143789,691.014960,0.94429811,2102.05,0.33106,0.3506,3,1,18.93154465,5.574042,78.37355779
19998,1271.408263,713.463760,0.93182902,1818.66,0.22698,0.2436,2,12,19.42797408,5.772102,114.99909150


In [29]:
# find rows having NaN
train.isnull().any(axis=0)

volume                  False
weight                  False
density                 False
surface_area            False
void_fraction           False
void_volume             False
metal_linker            False
organic_linker1         False
CO2/N2_selectivity      False
heat_adsorption_CO2     False
CO2_working_capacity    False
dtype: bool

In [30]:
# find row having inf
np.isinf(train).any(axis=0)

volume                  False
weight                  False
density                 False
surface_area            False
void_fraction           False
void_volume             False
metal_linker            False
organic_linker1         False
CO2/N2_selectivity      False
heat_adsorption_CO2     False
CO2_working_capacity    False
dtype: bool

## 3. Prepare training and test sets

Prepare input
- x_train = train input
- y_train = train output
- x_test = test input
- y_test = test predict

In [31]:
ratio = 0.8
train_size = math.floor(train.shape[0]*0.8)
test_size = train.shape[0] - train_size
print("dataset size:", train.shape[0])
print("train size:", train_size)
print("test size:", test_size)

dataset size: 20000
train size: 16000
test size: 4000


In [32]:
feat = list(train.columns.values)
feat = {k: v for k, v in enumerate(feat)}
feat

{0: 'volume',
 1: 'weight',
 2: 'density',
 3: 'surface_area',
 4: 'void_fraction',
 5: 'void_volume',
 6: 'metal_linker',
 7: 'organic_linker1',
 8: 'CO2/N2_selectivity',
 9: 'heat_adsorption_CO2',
 10: 'CO2_working_capacity'}

In [33]:
x_train = train.iloc[0:train_size, :-1].astype(np.float32)
y_train = train.iloc[0:train_size, -1].astype(np.float32)
x_test = train.iloc[train_size:train_size+test_size, :-1].astype(np.float32)
y_true = train.iloc[train_size:train_size+test_size, -1].astype(np.float32)

## 4. Neural network

### 4.1: Feed forward neural network

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [35]:
# 1. creat model using sequential method.
model = Sequential()
model.add(Dense(12, input_dim=(x_train.shape[1]), activation='relu')) # input
model.add(Dense(24, activation='relu')) # hidden 1
model.add(Dense(12, activation='relu')) # hidden 2
model.add(Dense(1, activation='linear')) # output

In [36]:
# 2. compile model
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
# 2. train model
model.fit(x_train, y_train, epochs=100, batch_size=64)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 73.6756 - accuracy: 0.0000e+00
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 67.8083 - accuracy: 0.0000e+00
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 63.7202 - accuracy: 0.0000e+00
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 57.0864 - accuracy: 0.0000e+00
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 43.7778 - accuracy: 0.0000e+00
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 40.4220 - accuracy: 0.0000e+00
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 38.9782 - accuracy: 0.0000e+00
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 38.6508 - accuracy: 0.0000e+00
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 37.4780 - accuracy: 0.0000e+00
Epoch 10/100
250/250 [========================

250/250 [==============================] - 0s 2ms/step - loss: 33.0979 - accuracy: 0.0000e+00
Epoch 78/100
250/250 [==============================] - 0s 2ms/step - loss: 32.7318 - accuracy: 0.0000e+00
Epoch 79/100
250/250 [==============================] - 0s 2ms/step - loss: 33.1344 - accuracy: 0.0000e+00
Epoch 80/100
250/250 [==============================] - 0s 2ms/step - loss: 32.6052 - accuracy: 0.0000e+00
Epoch 81/100
250/250 [==============================] - 0s 2ms/step - loss: 32.7065 - accuracy: 0.0000e+00
Epoch 82/100
250/250 [==============================] - 0s 2ms/step - loss: 32.3343 - accuracy: 0.0000e+00
Epoch 83/100
250/250 [==============================] - 0s 2ms/step - loss: 32.6530 - accuracy: 0.0000e+00
Epoch 84/100
250/250 [==============================] - 0s 2ms/step - loss: 32.4171 - accuracy: 0.0000e+00
Epoch 85/100
250/250 [==============================] - 0s 2ms/step - loss: 32.7507 - accuracy: 0.0000e+00
Epoch 86/100
250/250 [============================

<details>
<summary> <font color='green'>Click here for some more information about hyperparaper of neural network</font></summary>
We use MAE as a loss function in the neural network but we use LMAE as a metric in our competition. Is this reasonable?
</details>

In [37]:
y_pred = model.predict(x_test)
y_pred

array([[175.2972  ],
       [ 75.635796],
       [328.79987 ],
       ...,
       [102.01535 ],
       [120.91713 ],
       [193.30069 ]], dtype=float32)

In [38]:
log_mae = np.log10(mean_absolute_error(y_pred, y_true))
log_mae

1.4920379

## 4.2 Customization

In [39]:
import tensorflow.keras.backend as kb

def custom_loss(y_actual, y_pred): 
    custom_loss=tf.experimental.numpy.log10(kb.sum(kb.abs(y_actual - y_pred)) / y_actual.shape[0])
    return custom_loss

In [40]:
model.compile(loss=custom_loss, optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=64)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 1.5084 - accuracy: 0.0000e+00
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5090 - accuracy: 0.0000e+00
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5049 - accuracy: 0.0000e+00
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5087 - accuracy: 0.0000e+00
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5028 - accuracy: 0.0000e+00
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5038 - accuracy: 0.0000e+00
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5063 - accuracy: 0.0000e+00
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5045 - accuracy: 0.0000e+00
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5007 - accuracy: 0.0000e+00
Epoch 10/100
250/250 [==============================] -

250/250 [==============================] - 0s 2ms/step - loss: 1.4969 - accuracy: 0.0000e+00
Epoch 79/100
250/250 [==============================] - 0s 2ms/step - loss: 1.4968 - accuracy: 0.0000e+00
Epoch 80/100
250/250 [==============================] - 0s 2ms/step - loss: 1.4933 - accuracy: 0.0000e+00
Epoch 81/100
250/250 [==============================] - 0s 2ms/step - loss: 1.4986 - accuracy: 0.0000e+00
Epoch 82/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5002 - accuracy: 0.0000e+00
Epoch 83/100
250/250 [==============================] - 0s 2ms/step - loss: 1.4941 - accuracy: 0.0000e+00
Epoch 84/100
250/250 [==============================] - 0s 2ms/step - loss: 1.4939 - accuracy: 0.0000e+00
Epoch 85/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5015 - accuracy: 0.0000e+00
Epoch 86/100
250/250 [==============================] - 0s 2ms/step - loss: 1.4958 - accuracy: 0.0000e+00
Epoch 87/100
250/250 [==============================] - 0s 

In [ ]:
# predict
y_pred = model.predict(x_test)
# evaluate error
log_mae = np.log10(mean_absolute_error(y_pred, y_true))
log_mae

## 5. Predicting CO<sub>2</sub> WC 

### Import pretest and proprocessing

Today we will prepare a submission file for pretest set for the phase 1 (Development).

In [ ]:
pretest = pd.read_csv(dataset + "pretest.csv")
pretest.shape

In [ ]:
col = ["functional_groups", "topology"]
for i in col:
    pretest[i] = pretest[i].astype("category").cat.codes

pretest.dtypes

In [ ]:
pretest

### Let's predict and create a submission file

Join the [Codalab competition](https://competitions.codalab.org/competitions/34540) for this course!

Create a `submission.csv` with your predictions to join the competition and upload it to the competition site.

In [ ]:
pretest_pred = model.predict(pretest.iloc[:, [1,2,3,4,5,7,8,9,11,12]])
pretest_pred

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=71f6eb72-f79d-4dec-bd79-97456fb0fc4e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>